https://www.youtube.com/watch?v=qWESWDKYWFU

In [ ]:
static StaticJsonDocument<256> config;

void setup() {
  Serial.begin(9600);
}

void updateConfig() {
  if (Serial.available() > 0) {
    String value = Serial.readString();
      Serial.println(value.c_str());
    if (value == "show") {
      serializeJsonPretty(config, Serial);
      Serial.println();
    } else{
      const auto deser_err = deserializeJson(config, Serial);
 
      if (!deser_err) {
        float clutchFactor   = config["clutch_smoothing_factor"].as<float>();
        float breakFactor    = config["break_smoothing_factor"].as<float>();
        float throttleFactor = config["throttle_smoothing_factor"].as<float>();

        clutchFilter   = new Ewma(clutchFactor);
        breakFilter    = new Ewma(breakFactor);
        throttleFilter = new Ewma(throttleFactor);
      }
    }
  }
}

In [1]:
import serial

In [60]:
import sys
import glob
import serial

class SerialPort:
    def __init__(self, port):
        self.serial_port = serial.Serial(port, 9600, timeout=2)


    def send(self, value):
        self.serial_port.write(value)

    def receive(self):
        result = ''
        value = self.serial_port.readline()
        while(len(value) >0):
            result += value.decode('ascii')
            value = self.serial_port.readline()
        return result
    
    def close(self):
        self.serial_port.close()
    
    @staticmethod
    def avaiable():
        """ Lists serial port names
    
            :raises EnvironmentError:
                On unsupported or unknown platforms
            :returns:
                A list of the serial ports available on the system
        """
        if sys.platform.startswith('win'):
            ports = ['COM%s' % (i + 1) for i in range(256)]
        elif sys.platform.startswith('linux') or sys.platform.startswith('cygwin'):
            # this excludes your current terminal "/dev/tty"
            ports = glob.glob('/dev/tty*')
        elif sys.platform.startswith('darwin'):
            ports = glob.glob('/dev/tty.*')
        else:
            raise EnvironmentError('Unsupported platform')
    
        result = []
        for port in ports:
            try:
                s = serial.Serial(port)
                s.close()
                result.append(port)
            except (OSError, serial.SerialException):
                pass
        return result

In [61]:
SerialPort.avaiable()

['/dev/ttyACM0', '/dev/ttyS0']

In [62]:
serial_port = SerialPort(SerialPort.avaiable()[0])

In [63]:
serial_port.send(b'''
{
    "clutch_smoothing_factor": 0.9,
    "break_smoothing_factor": 0.9,
    "throttle_smoothing_factor": 0.9
}
''')

In [64]:
serial_port.send(b'show')

In [65]:
print(serial_port.receive())

null

In [59]:
serial_port.close()